# Image-Basic-Recognition_static

O Sistema deve identificar as provas a serem analisadas, independente de sua posição

#### [1]Desta forma haverá um delimitador na câmera, onde analizará apenas as informações contidas em seu espaço

#### [2]Desta forma após identificar o padrão, irá contabilizar 3 segundos para salvar uma cópia

Alinhando a prova

In [24]:
import cv2
import numpy as np
 
#função responsável por alinhar as imagens baseado no template pré-configurado
def align_images(im1, im2):
    MAX_FEATURES = 500
    GOOD_MATCH_PERCENT = 0.15
    
    #Alterando imagens para escola cinza
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    
    #Detectando características do ORB e calculando descritores.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
    
    # Ocorrências de características
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Ordenando melhores ocorrências
    matches.sort(key=lambda x: x.distance, reverse=False)
    
    # Removendo ocorrências que não possuem melhores porcentagem de reconhecimento
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
    
    # Draw melhores matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imshow('imMatches', imMatches)
    cv2.waitKey(0)
    
    
    # Extraindo localização de melhores correspondências
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # buscando homografia
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Usando homografia
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

# Read reference image
refFilename = "dataset/imgs/gabarito_template_geral2.png"
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

# Lendo Imagem a alinhar
imFilename = "dataset/imgs/gabarito_template_geral_preenchido_flip_180.png"
im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

# A imagem alinhada será restaurada na imReg. 
# A homografia estimada será armazenada em h.
imReg, h = align_images(im, imReference)

cv2.imshow('aligned', imReg)
cv2.waitKey(0)

print("Homografia Estimada : \n",  h)
cv2.destroyAllWindows()

Homografia Estimada : 
 [[-1.00020172e+00  1.15150980e-04  4.84021631e+02]
 [ 4.97546691e-05 -1.00027973e+00  3.18083266e+02]
 [ 6.60252792e-07  8.52319484e-07  1.00000000e+00]]


### Identificando respostas do gabarito

In [1]:
import cv2
import numpy as np

def answer(ponto_ocupacao):
    global width
    ocupacao = round((ponto_ocupacao*100)/width)
    if ocupacao < 15: return
    if ocupacao < 30: return 'A'
    if ocupacao < 50: return 'B'
    if ocupacao < 70: return 'C'
    if ocupacao < 90: return 'D'
    return 'E'

def coordinates_question(img):

    # Criando o detector baseado na versão do CV
    is_cv3 = cv2.__version__.startswith("3.")
    if is_cv3:
        detector = cv2.SimpleBlobDetector_create()
    else:
        detector = cv2.SimpleBlobDetector()
     
    #Detectando corpos
    keypoints = detector.detect(img)    
    # Criando nova imagem com os elementos identificandos pelo detector
    im_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]),(0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow("teste",im_with_keypoints)
    cv2.waitKey(0)
    return keypoints

def evaluate_question(image):
    
    #global template
    
    coordinates = coordinates_question(image)  
    response = []
    if coordinates:
        for point in coordinates:            
            response.append(answer(point.pt[0]))
    im_with_keypoints = cv2.drawKeypoints(template, coordinates, np.array([]),(0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return response


if __name__ == "__main__":
    image = cv2.imread('dataset/imgs/gabarito_template_geral_preenchido.png')
    template = cv2.imread('dataset/imgs/gabarito_template_geral_alternativas.png',0)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    
    template = cv2.adaptiveThreshold(template,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    
    cv2.imshow("template",template)
    cv2.waitKey(0)
    
    cv2.imshow("gray",gray)
    cv2.waitKey(0)
    result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    height, width = template.shape[:2]    
    #Create Bounding Box
    top_left = max_loc
    bottom_right = (top_left[0] + width, top_left[1] + height)
    #cv2.rectangle(image, top_left, bottom_right, (0,0,255), 2)
    answer_values = {}
    for i in range(0,10):
        bottom_right = (bottom_right[0], bottom_right[1] + height)
        top_left = (top_left[0],top_left[1]+height)
        cv2.rectangle(image, top_left, bottom_right, (0,0,255), 2)
        
        cv2.imshow("image",image)
        cv2.waitKey(0)
        
        cropped = image[top_left[1]:bottom_right[1],top_left[0]:bottom_right[0]]

        answer_values[i+1] = evaluate_question(cropped)

    print(answer_values)
    cv2.destroyAllWindows()

error: OpenCV(3.4.1) /io/opencv/modules/features2d/src/draw.cpp:115: error: (-215) !outImage.empty() in function drawKeypoints
